In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page containing the list of cities
url = 'https://www.britannica.com/topic/list-of-cities-and-towns-in-India-2033033'

# Send a request to the website
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all city names
city_elements = soup.find_all('a', class_='md-crosslink')

# Extract city names
cities = [city.get_text() for city in city_elements]

# Save the city names to a CSV file
cities_df = pd.DataFrame(cities, columns=['City'])
cities_df.to_csv('all_india_cities.csv', index=False)


genrating data

In [ ]:
import pandas as pd
import random

# Load country and city data from the uploaded CSV file
file_path = 'country_data.csv'
df_world = pd.read_csv(file_path)

brands_df = pd.read_csv('brands_only (1).csv')
brands = brands_df['Brand'].tolist()
# Define brand categories
brand_categories = [
    'Film & Animation', 'Autos & Vehicles', 'Music', 'Pets & Animals', 'Sports', 'Short Movies', 'Travel & Events',
    'Gaming', 'Videoblogging', 'People & Blogs', 'Comedy', 'Entertainment', 'News & Politics', 'Howto & Style',
    'Education', 'Science & Technology', 'Nonprofits & Activism', 'Movies', 'Anime/Animation', 'Action/Adventure',
    'Classics', 'Comedy', 'Documentary', 'Drama', 'Family', 'Foreign', 'Horror', 'Sci-Fi/Fantasy', 'Thriller', 'Shorts',
    'Shows', 'Trailers'
]

# Function to generate random view percentages for countries
def generate_country_view_percentages(num_countries):
    percentages = [random.randint(10, 50) for _ in range(num_countries)]
    total = sum(percentages)
    normalized_percentages = [round(p / total * 100, 2) for p in percentages]
    return normalized_percentages

# # Sample brands (replace with actual brand names)
# brands = ['Brand1', 'Brand2', 'Brand3']

# Generate the dataset
data = []
max_rows_per_file = 100000  # Adjust this value based on your requirements

file_counter = 1
row_counter = 0

for brand in brands:
    for brand_category in brand_categories:
        countries = df_world['country'].unique()
        country_percentages = generate_country_view_percentages(len(countries))
        
        for country, total_percentage in zip(countries, country_percentages):
            country_cities = df_world[df_world['country'] == country]['city'].tolist()
            num_cities = len(country_cities)
            if num_cities > 0:
                # Generate random percentages for cities that sum up to the country's percentage
                city_percentages = [random.uniform(0.5, 1.5) for _ in range(num_cities)]
                city_percentages_total = sum(city_percentages)
                normalized_city_percentages = [round((p / city_percentages_total) * total_percentage, 2) for p in city_percentages]

                # Adjust the first city percentage to ensure the total percentage is correct
                diff = total_percentage - sum(normalized_city_percentages)
                normalized_city_percentages[0] += diff

                for city, percentage in zip(country_cities, normalized_city_percentages):
                    row = [brand, brand_category, country, city, percentage]
                    data.append(row)
                    row_counter += 1
                    
                    # Check if the number of rows exceeds the limit
                    if row_counter >= max_rows_per_file:
                        # Save the current dataset to a CSV file
                        columns = ['Brand', 'Brand Category', 'Country', 'City', 'View Percentage']
                        dummy_df = pd.DataFrame(data, columns=columns)
                        dummy_df.to_csv(f'dummy_city_view_percentages_part_{file_counter}.csv', index=False)
                        
                        # Reset data and counters
                        data = []
                        row_counter = 0
                        file_counter += 1

# Save any remaining data to a CSV file
if data:
    columns = ['Brand', 'Brand Category', 'Country', 'City', 'View Percentage']
    dummy_df = pd.DataFrame(data, columns=columns)
    dummy_df.to_csv(f'dummy_city_view_percentages_part_{file_counter}.csv', index=False)


preprocessing-data-1

In [ ]:
import pandas as pd
import random
import os

# Function to distribute a country's percentage among its cities
def distribute_percentage_among_cities(total_percentage, num_cities):
    if num_cities == 1:
        return [total_percentage]
    
    # Create random values and normalize them to sum up to 1
    random_values = [random.uniform(0.5, 1.5) for _ in range(num_cities)]
    random_total = sum(random_values)
    
    # Assign city percentages based on random values, scaled to the total_percentage
    city_percentages = [round((value / random_total) * total_percentage, 2) for value in random_values]
    
    # Adjust the last city's percentage to ensure the sum equals the total_percentage
    difference = total_percentage - sum(city_percentages)
    city_percentages[-1] += difference
    
    return city_percentages

# Directory containing the generated CSV files
directory = "dataset_for_country_cities"  # Replace with the actual path to your 290 files

# Process each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        print(f"Processing file: {file_path}")
        
        # Load the data
        df = pd.read_csv(file_path)
        
        # Iterate over each combination of Brand, Brand Category, and Country
        updated_rows = []
        for (brand, brand_category, country), group in df.groupby(['Brand', 'Brand Category', 'Country']):
            # Assign a random percentage to the country for the specific brand and category
            country_percentage = random.randint(10, 50)
            num_cities = len(group)
            
            # Distribute the country percentage among the cities
            city_percentages = distribute_percentage_among_cities(country_percentage, num_cities)
            
            for i, (index, row) in enumerate(group.iterrows()):
                row['Country Percentage'] = country_percentage
                row['City Percentage'] = city_percentages[i]
                updated_rows.append(row)
        
        # Save the updated data back to the file
        updated_df = pd.DataFrame(updated_rows)
        updated_df.to_csv(file_path, index=False)

print("City and country percentage columns added and saved for all files.")


encodeing Data (Prepreocessing -step-2)

In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import joblib

# Directory containing the generated CSV files
directory = "dataset_for_country_cities"  # Replace with the actual path to your 290 files

# Initialize an empty list to store all the data for fitting the encoder
all_data = []

# Load all data to fit the encoder
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        print(f"Loading data for encoder fitting: {file_path}")
        df = pd.read_csv(file_path)
        all_data.append(df[['Brand', 'Brand Category', 'Country', 'City', 'Country Percentage']])

# Combine all the data into a single DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder on the entire dataset
encoder.fit(df_all)

# Save the encoder for later use
encoder_path = 'ml_models/models/cities_model/encoder.pkl'
joblib.dump(encoder, encoder_path)
print(f"Encoder saved to {encoder_path}")


features and target PreProcessing

In [ ]:
import pandas as pd
import os
import joblib
import scipy.sparse

# Directory containing the generated CSV files
directory = "dataset_for_country_cities"  # Replace with the actual path to your 290 files

# Define the chunk size
chunk_size = 50000  # Adjust this value based on your memory limitations

# Initialize empty lists to store encoded chunks and target chunks
encoded_chunks = []
target_chunks = []

# Process each file in the directory and encode using the fitted encoder
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        print(f"Processing file: {file_path}")
        
        # Load the data in chunks
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            X = chunk[['Brand', 'Brand Category', 'Country', 'City', 'Country Percentage']]
            y = chunk['City Percentage']
            
            # Transform the features using the already fitted encoder
            encoded_chunk = encoder.transform(X)
            encoded_chunks.append(encoded_chunk)
            target_chunks.append(y)

# Save the encoded chunks and target chunks separately
encoded_data_path = 'ml_models/models/cities_model/encoded_features.pkl'
target_data_path = 'ml_models/models/cities_model/target.pkl'
joblib.dump(encoded_chunks, encoded_data_path)
joblib.dump(target_chunks, target_data_path)
print(f"Encoded data saved to {encoded_data_path}")
print(f"Target data saved to {target_data_path}")


Model Training

In [ ]:
import joblib
import scipy.sparse
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load the encoded features and targets
encoded_data_path = 'ml_models/models/cities_model/encoded_features.pkl'
target_data_path = 'ml_models/models/cities_model/target.pkl'

encoded_chunks = joblib.load(encoded_data_path)
target_chunks = joblib.load(target_data_path)

# Combine the encoded chunks and target chunks
X = scipy.sparse.vstack(encoded_chunks)  # Keep it as a sparse matrix
y = pd.concat(target_chunks, ignore_index=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SGDRegressor model
model = SGDRegressor(
    max_iter=100,       # Single pass through the data for each chunk
    tol=None,         # Disable early stopping (for demonstration)
    random_state=42
)

# # Chunked training approach
# chunk_size = 10000  # Adjust based on your memory capacity
# n_chunks = X_train.shape[0] // chunk_size

# for i in range(n_chunks):
#     start = i * chunk_size
#     end = start + chunk_size
#     X_chunk = X_train[start:end]
#     y_chunk = y_train[start:end]

#     # Fit the model on the chunk using partial_fit
#     model.partial_fit(X_chunk, y_chunk)

# Train the model on the entire dataset
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"Model R^2 score on test data: {score:.4f}")

# Save the trained model
model_path = 'ml_models/models/cities_model/city_percentage_model.pkl'
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")


In [ ]:
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import scipy.sparse

# Load the encoded features and targets
encoded_data_path = 'ml_models/models/cities_model/encoded_features.pkl'
target_data_path = 'ml_models/models/cities_model/target.pkl'

encoded_chunks = joblib.load(encoded_data_path)
target_chunks = joblib.load(target_data_path)

# Combine the encoded chunks and target chunks
X = scipy.sparse.vstack(encoded_chunks)  # Assuming the data is sparse
y = pd.concat(target_chunks, ignore_index=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"Model R^2 score on test data: {score:.4f}")

# Save the trained model
model_path = 'ml_models/models/cities_model/city_percentage_model.pkl'
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")


not final


not final

In [ ]:
import pandas as pd
import random

# Load the list of cities
cities_df = pd.read_csv('all_india_cities.csv')
cities = cities_df['City'].tolist()

# List of brand categories
brand_categories = [
    'Film & Animation', 'Autos & Vehicles', 'Music', 'Pets & Animals', 'Sports', 'Short Movies', 'Travel & Events',
    'Gaming', 'Videoblogging', 'People & Blogs', 'Comedy', 'Entertainment', 'News & Politics', 'Howto & Style',
    'Education', 'Science & Technology', 'Nonprofits & Activism', 'Movies', 'Anime/Animation', 'Action/Adventure',
    'Classics', 'Comedy', 'Documentary', 'Drama', 'Family', 'Foreign', 'Horror', 'Sci-Fi/Fantasy', 'Thriller', 'Shorts',
    'Shows', 'Trailers'
]

# Load the brands from the provided CSV file
brands_df = pd.read_csv('brands_only (1).csv')
brands = brands_df['Brand'].tolist()

# Function to generate random view percentages
def generate_view_percentages():
    percentages = [random.randint(5, 30) for _ in range(len(cities))]
    total = sum(percentages)
    return [round(p / total * 100, 2) for p in percentages]

# Generate the dummy dataset
data = []

for brand in brands:
    brand_category = random.choice(brand_categories)
    percentages = generate_view_percentages()
    for city, percentage in zip(cities, percentages):
        row = [brand, brand_category, city, percentage]
        data.append(row)

# Create a DataFrame
columns = ['Brand', 'Brand Category', 'City', 'View Percentage']
dummy_df = pd.DataFrame(data, columns=columns)

# Save the dummy dataset to a CSV file
dummy_df.to_csv('dummy_city_view_percentages_with_all_cities.csv', index=False)


In [27]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import joblib

# Load the generated dataset
df = pd.read_csv('dummy_city_view_percentages_with_all_cities.csv')

# Split the data into smaller chunks
chunk_size = 50000  # Adjust the chunk size based on your memory limitations
num_chunks = len(df) // chunk_size + 1

# Initialize the OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(df[['Brand', 'Brand Category', 'City']])

# Save the encoder
joblib.dump(encoder, 'ml_models/models/cities_model/encoder.pkl')

# Process each chunk separately
for i in range(num_chunks):
    print(f"Processing chunk {i+1}/{num_chunks}...")
    chunk = df[i*chunk_size:(i+1)*chunk_size]
    
    # Encode the categorical features
    encoded_features = encoder.transform(chunk[['Brand', 'Brand Category', 'City']])
    
    # Save the encoded features and target
    joblib.dump(encoded_features, f'ml_models/models/cities_model/encoded_features_chunk_{i+1}.pkl')
    joblib.dump(chunk['View Percentage'], f'ml_models/models/cities_model/target_chunk_{i+1}.pkl')

print("Data encoding and saving complete.")

Processing chunk 1/22...
Processing chunk 2/22...
Processing chunk 3/22...
Processing chunk 4/22...
Processing chunk 5/22...
Processing chunk 6/22...
Processing chunk 7/22...
Processing chunk 8/22...
Processing chunk 9/22...
Processing chunk 10/22...
Processing chunk 11/22...
Processing chunk 12/22...
Processing chunk 13/22...
Processing chunk 14/22...
Processing chunk 15/22...
Processing chunk 16/22...
Processing chunk 17/22...
Processing chunk 18/22...
Processing chunk 19/22...
Processing chunk 20/22...
Processing chunk 21/22...
Processing chunk 22/22...
Data encoding and saving complete.


In [28]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

# Initialize the SGDRegressor model
model = SGDRegressor()

# Train separate models on each chunk
for i in range(num_chunks):
    print(f"Training on chunk {i+1}/{num_chunks}...")
    X_chunk = joblib.load(f'ml_models/models/cities_model/encoded_features_chunk_{i+1}.pkl')
    y_chunk = joblib.load(f'ml_models/models/cities_model/target_chunk_{i+1}.pkl')
    
    model.partial_fit(X_chunk, y_chunk)
    
    # Evaluate the model on the same chunk
    y_pred = model.predict(X_chunk)
    mse = mean_squared_error(y_chunk, y_pred)
    print(f'Chunk {i+1} - Mean Squared Error: {mse}')

# Save the final model
joblib.dump(model, 'ml_models/models/cities_model/sgd_view_percentage_model.pkl')

print("Model training complete and saved.")


Training on chunk 1/22...
Chunk 1 - Mean Squared Error: 0.006640443061600881
Training on chunk 2/22...
Chunk 2 - Mean Squared Error: 0.006484119164189637
Training on chunk 3/22...
Chunk 3 - Mean Squared Error: 0.006301960383437895
Training on chunk 4/22...
Chunk 4 - Mean Squared Error: 0.0062875247738416375
Training on chunk 5/22...
Chunk 5 - Mean Squared Error: 0.006266396908099242
Training on chunk 6/22...
Chunk 6 - Mean Squared Error: 0.006211923130502211
Training on chunk 7/22...
Chunk 7 - Mean Squared Error: 0.006263228255113702
Training on chunk 8/22...
Chunk 8 - Mean Squared Error: 0.006263741279470197
Training on chunk 9/22...
Chunk 9 - Mean Squared Error: 0.006245383318982498
Training on chunk 10/22...
Chunk 10 - Mean Squared Error: 0.0063311014224954635
Training on chunk 11/22...
Chunk 11 - Mean Squared Error: 0.0062543392603962395
Training on chunk 12/22...
Chunk 12 - Mean Squared Error: 0.006225541454597793
Training on chunk 13/22...
Chunk 13 - Mean Squared Error: 0.0063168

In [30]:
import joblib
import pandas as pd
import numpy as np

# Load the trained model and encoder
model = joblib.load('ml_models/models/cities_model/sgd_view_percentage_model.pkl')
encoder = joblib.load('ml_models/models/cities_model/encoder.pkl')

# List of major cities in India
cities = [
    'Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Ahmedabad', 'Chennai', 'Kolkata', 'Surat', 'Pune', 'Jaipur',
    # Add more cities as needed
]

# Function to predict view percentages for a new brand
def predict_view_percentages(brand_name, brand_category, cities, model, encoder):
    try:
        # Create a DataFrame with the new brand data
        new_data = pd.DataFrame({
            'Brand': [brand_name] * len(cities),
            'Brand Category': [brand_category] * len(cities),
            'City': cities
        })

        # Encode the categorical features
        encoded_features = encoder.transform(new_data)
        
        # Predict view percentages
        predictions = model.predict(encoded_features)
        
        # Create a DataFrame with the results
        result_df = pd.DataFrame({
            'City': cities,
            'View Percentage': predictions
        })
        
        return result_df
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None

# Example usage
brand_name = 'Puma'
brand_category = 'Sports'
predicted_percentages = predict_view_percentages(brand_name, brand_category, cities, model, encoder)
print(predicted_percentages)


        City  View Percentage
0     Mumbai         0.182020
1      Delhi         0.184602
2  Bangalore         0.175156
3  Hyderabad         0.182778
4  Ahmedabad         0.175156
5    Chennai         0.181023
6    Kolkata         0.182982
7      Surat         0.182214
8       Pune         0.180695
9     Jaipur         0.184509
